In [1]:
!pip install pandas polars bs4 openpyxl pyarrow

import pandas as pd
import polars as pl
import bs4
import requests
import re
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 3.0 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for pyarrow from https://files.pythonhosted.org/packages/a1/14/4ffed5e85b96f0c0ae9e026f940bf71ac7dfbfbffff9f3fe339e32bfce2c/pyarrow-13.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata
Using cached pyarrow-13.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (40.0 MB)


In [2]:
# get all links that contains .csv 
relative_path = './levantamento_semanal_precos/'
url = 'https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/levantamento-de-precos-de-combustiveis-ultimas-semanas-pesquisadas'
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
links = soup.find_all('a', href=re.compile(r'revendas'))
# get only links between href and " target
links = [link['href'] for link in links if 'target' in link.attrs]

links
# # download all files using multiprocessing and urllib
import urllib.request
from multiprocessing.pool import ThreadPool

def download_file(link):
    # from string https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/pmqc/2023/pmqc_2023_04.csv
    # get year of file 
    year = link.split('/')[-2]
    name_file = link.split('/')[-1]
    if not os.path.exists(f"./levantamento_semanal_precos/"):
        os.makedirs(f"./levantamento_semanal_precos/")
    try:
        urllib.request.urlretrieve(link, f"./levantamento_semanal_precos/{name_file}")
        print(f"Downloaded {name_file}")
    except:
        print(f"Error to download {name_file}")

pool = ThreadPool(8)
pool.map(download_file, links)
pool.close()



Downloaded revendas_lpc_2023-05-28_2023-06-03.xlsx
Downloaded revendas_lpc_2023-07-09_2023-07-15.xlsx
Downloaded revendas_lpc_2023-06-25_2023-07-01.xlsx
Downloaded revendas_lpc_2023-05-14_2023-05-20.xlsx
Downloaded revendas_lpc_2023-06-11_2023-06-17.xlsx
Downloaded revendas_lpc_2023-09-17_2023-09-23.xlsx
Downloaded revendas_lpc_2023-08-06_2023-08-12.xlsx
Downloaded revendas_lpc_2023-05-21_2023-05-27.xlsx
Downloaded revendas_lpc_2023-05-07_2023-05-13.xlsx
Downloaded revendas_lpc_2023-07-02_2023-07-08.xlsx
Downloaded revendas_lpc_2023-07-23_2023-07-29.xlsx
Downloaded revendas_lpc_2023-06-04_2023-06-10.xlsx
Downloaded revendas-lpc-3004-0605.xlsx
Downloaded revendas-lpc-2023-04-02-2023-04-08.xlsx
Downloaded revendas-lpc-16-2204.xlsx
Downloaded revendas_lpc_2023-07-16_2023-07-22.xlsx
Downloaded revendas_lpc_2023-06-18_2023-06-24.xlsx
Downloaded revendas_lpc_2023-03-19_2023-03-25.xlsx
Downloaded revendas_lpc_2023-09-10_2023-09-16.xlsx
Downloaded revendas_lpc_2023-07-30_2023-08-05.xlsx
Downlo

FileExistsError: [Errno 17] File exists: './levantamento_semanal_precos/'

In [3]:
# read all execels inside folder and concat to polars dataframe

# get all files inside folder
files = os.listdir(relative_path)
# get only files that contains .xlsx
files = [file for file in files if '.xlsx' in file]
# read all files and concat to dataframe

df = pd.DataFrame() 
for file in files:
    try:
        df = pd.concat([df, pd.read_excel(f"{relative_path}{file}", engine='openpyxl')])
    except:
        pass
df


/home/mfelipemota/projects/olho-no-combustivel/.venv/lib/python3.10/site-packages/openpyxl/reader/workbook.py:84: UserWarning: File contains an invalid specification for 0. This will be removed
  warn(msg)
/home/mfelipemota/projects/olho-no-combustivel/.venv/lib/python3.10/site-packages/openpyxl/reader/workbook.py:102: UserWarning: Defined names for sheet index 0 cannot be located
  warn(f"Defined names for sheet index {idx} cannot be located")
/home/mfelipemota/projects/olho-no-combustivel/.venv/lib/python3.10/site-packages/openpyxl/reader/workbook.py:84: UserWarning: File contains an invalid specification for 0. This will be removed
  warn(msg)
/home/mfelipemota/projects/olho-no-combustivel/.venv/lib/python3.10/site-packages/openpyxl/reader/workbook.py:102: UserWarning: Defined names for sheet index 0 cannot be located
  warn(f"Defined names for sheet index {idx} cannot be located")
/home/mfelipemota/projects/olho-no-combustivel/.venv/lib/python3.10/site-packages/openpyxl/reader/work

,"AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,SUPERINTENDÊNCIA DE DEFESA DA CONCORRÊNCIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SISTEMA DE LEVANTAMENTO DE PREÇOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INTERVALO DE TEMPO: SEMANAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PERÍODO: 19/02/2023 A 25/02/2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22834,43026473000170,AUTO POSTO BALI LTDA,NaN,AV MADRID,1795,QUADRA146 LOTE 01,LOT FAICALVILLE,74350730,GOIANIA,GOIAS,IPIRANGA,GASOLINA ADITIVADA,R$ / litro,5.77,2023-07-18 00:00:00
22835,43026473000170,AUTO POSTO BALI LTDA,NaN,AV MADRID,1795,QUADRA146 LOTE 01,LOT FAICALVILLE,74350730,GOIANIA,GOIAS,IPIRANGA,DIESEL S10,R$ / litro,5.19,2023-07-18 00:00:00
22836,43026473000170,AUTO POSTO BALI LTDA,NaN,AV MADRID,1795,QUADRA146 LOTE 01,LOT FAICALVILLE,74350730,GOIANIA,GOIAS,IPIRANGA,ETANOL,R$ / litro,3.97,2023-07-18 00:00:00
22837,46478162000130,L E M COMERCIO DE GAS LTDA,NaN,RUA INÁCIO VALE,248,NaN,ALTO DE SÃO MANOEL,59631270,MOSSORO,RIO GRANDE DO NORTE,BAHIANA,GLP,R$ / 13 kg,120,2023-07-17 00:00:00


In [ ]:
import pyarrow

df_comb = df.copy()
## filter column Unnamed 7 is int
df_comb = df_comb[df_comb['Unnamed: 7'].apply(lambda x: isinstance(x, int))]
df_comb.columns = [
    'CNPJ',
    'RAZAO_SOCIAL',
    'NOME_FANTASIA',
    'ENDERECO',
    'NUMERO',
    'COMPLEMENTO',
    'BAIRRO',
    'CEP',
    'MUNICIPIO',
    'ESTADO',
    'BANDEIRA',
    'PRODUTO',
    'UNIDADE_PRECO',
    'PRECO',
    'DATA_COLETA',
]

df_comb.columns = [col.lower() for col in df_comb.columns]

df_comb = df_comb[df_comb['estado'] == 'MATO GROSSO DO SUL']

# convert to polars
df_comb = pl.from_pandas(df_comb.astype(str))
# sort by DATA_COLETA desc in polars
df_comb = df_comb.sort('data_coleta', descending=True)
# covert column preco to double in polars
df_comb = df_comb.with_columns(
    pl.col('preco').cast(pl.Float64),
    pl.col('data_coleta').str.to_datetime("%Y-%m-%d 00:00:00"),
    pl.col('cnpj').str.replace(r'\D', '')
)
df_comb.write_json('fuel_price_processed/precos.json', row_oriented=True)